# Project 3, Part 5, Use geodesic distances and zip code and population data to enhance our BART model

University of California, Berkeley

Master of Information and Data Science (MIDS) program

w205 - Fundamentals of Data Engineering

Students in the group:
* Aris Chalini
* Jack Galvin
* Matt Lauritzen

Year: 2022

Semester: Spring

Section: 09


# Included Modules and Packages

Code cell containing your includes for modules and packages

Some starter code is provided

You may change the starter code as needed

You may add as much code and/or as many code cells as you need

In [1]:
import math
import numpy as np
import pandas as pd

import psycopg2

from geographiclib.geodesic import Geodesic

# Supporting code

Code cells containing any supporting code, such as connecting to the database, any functions, etc.  

Remember you can freely use any code from the labs. You do not need to cite code from the labs.

Some starter code is provided

You may change the starter code as needed

You may add as much code and/or as many code cells as you need

In [2]:
def my_calculate_box(point, miles):
    "Given a point and miles, calculate the box in form left, right, top, bottom"
    
    geod = Geodesic.WGS84

    kilometers = miles * 1.60934
    meters = kilometers * 1000

    g = geod.Direct(point[0], point[1], 270, meters)
    left = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 90, meters)
    right = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 0, meters)
    top = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 180, meters)
    bottom = (g['lat2'], g['lon2'])
    
    return(left, right, top, bottom)

In [3]:
def my_station_get_zips(station, miles):
    "given a station, pull all zip codes with miles distance, print them, sum the population"
    
    connection.rollback()
    
    query = "select latitude, longitude from stations "
    query += "where station = '" + station + "'"
    
    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
    for row in rows:
        latitude = row[0]
        longitude = row[1]
        
    point = (latitude, longitude)
        
    (left, right, top, bottom) = my_calculate_box(point, miles)
    
    query = "select zip, population from zip_codes "
    query += " where latitude >= " + str(bottom[0])
    query += " and latitude <= " + str(top [0])
    query += " and longitude >= " + str(left[1])
    query += " and longitude <= " + str(right[1])
    query += " order by 1 "

    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
    print("\n-------------------------------------------------------------------------------")
    print("  Zip Codes within " + str(miles) + " mile(s) of " + station + " BART Station")
    print("-------------------------------------------------------------------------------\n")
    
    total_population = 0
    
    for row in rows:
        zip = row[0]
        population = row[1]
        print("     zip:", zip, "  population: ", f'{population:10,}')
        total_population += population
        
    
    print("\n-------------------------------------------------------------------------------")
    print("  Total Population: ", f'{total_population:10,}')
    print("-------------------------------------------------------------------------------")

In [4]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [5]:
cursor = connection.cursor()

# 3.5.1 Find all zip codes, with population, within 1 mile of the Downtown Berkeley station

Our stations table has each station, along with the latitude and longitude point for the station

The function above my_station_get_zips() takes a station name and X miles (which can be decimal such as 1.5) and finds all zip codes whose geograpic center lies within a box X miles from the station along with the population and the total population

Zip code data is pulled from the zip_codes table

Week 2 has an optional module that covers geodesic distance calculation and how to use the box method




## Since this is the first one, a solution code cell is provided for you to execute and then pattern the rest after



In [6]:
my_station_get_zips("Downtown Berkeley", 1)


-------------------------------------------------------------------------------
  Zip Codes within 1 mile(s) of Downtown Berkeley BART Station
-------------------------------------------------------------------------------

     zip: 94702   population:      17,092
     zip: 94703   population:      21,937
     zip: 94704   population:      29,190
     zip: 94709   population:      11,740
     zip: 94720   population:       2,971

-------------------------------------------------------------------------------
  Total Population:      82,930
-------------------------------------------------------------------------------


# 3.5.2 Find all zip codes, with population, within 1, 2, 3, 4, and 5 miles of the Downtown Berkeley station

Write a loop to call the function my_station_get_zips() for each of 1, 2, 3, 4, and 5 miles




In [10]:
for i in range(1,6):
    my_station_get_zips("Downtown Berkeley", i)


-------------------------------------------------------------------------------
  Zip Codes within 1 mile(s) of Downtown Berkeley BART Station
-------------------------------------------------------------------------------

     zip: 94702   population:      17,092
     zip: 94703   population:      21,937
     zip: 94704   population:      29,190
     zip: 94709   population:      11,740
     zip: 94720   population:       2,971

-------------------------------------------------------------------------------
  Total Population:      82,930
-------------------------------------------------------------------------------

-------------------------------------------------------------------------------
  Zip Codes within 2 mile(s) of Downtown Berkeley BART Station
-------------------------------------------------------------------------------

     zip: 94618   population:      17,041
     zip: 94702   population:      17,092
     zip: 94703   population:      21,937
     zip: 94704   pop

# 3.5.3 Find all zip codes, with population, within 2 miles of  the Antioch station

Use the my_station_get_zips() function

In [9]:
my_station_get_zips("Antioch", 2)


-------------------------------------------------------------------------------
  Zip Codes within 2 mile(s) of Antioch BART Station
-------------------------------------------------------------------------------

     zip: 94509   population:      66,933

-------------------------------------------------------------------------------
  Total Population:      66,933
-------------------------------------------------------------------------------


# 3.5.4 Find all zip codes, with population, within 2 miles of  the Powell Street station

Use the my_station_get_zips() function

In [12]:
my_station_get_zips("Powell Street", 2)


-------------------------------------------------------------------------------
  Zip Codes within 2 mile(s) of Powell Street BART Station
-------------------------------------------------------------------------------

     zip: 94102   population:      31,067
     zip: 94103   population:      28,735
     zip: 94104   population:         546
     zip: 94105   population:       9,155
     zip: 94107   population:      29,689
     zip: 94108   population:      14,550
     zip: 94109   population:      56,677
     zip: 94111   population:       3,620
     zip: 94114   population:      34,754
     zip: 94115   population:      35,004
     zip: 94123   population:      25,941
     zip: 94133   population:      26,527
     zip: 94158   population:       7,291

-------------------------------------------------------------------------------
  Total Population:     303,556
-------------------------------------------------------------------------------


# 3.5.5 Find all zip codes, with population, within 2 miles of  the Coliseum station

Use the my_station_get_zips() function

In [13]:
my_station_get_zips("Coliseum", 2)


-------------------------------------------------------------------------------
  Zip Codes within 2 mile(s) of Coliseum BART Station
-------------------------------------------------------------------------------

     zip: 94601   population:      52,299
     zip: 94603   population:      34,593
     zip: 94613   population:         861
     zip: 94621   population:      35,287

-------------------------------------------------------------------------------
  Total Population:     123,040
-------------------------------------------------------------------------------


# 3.5.6 Find all zip codes, with population, within 2 miles of  the Milpitas station

Use the my_station_get_zips() function

In [14]:
my_station_get_zips("Milpitas", 2)


-------------------------------------------------------------------------------
  Zip Codes within 2 mile(s) of Milpitas BART Station
-------------------------------------------------------------------------------

     zip: 95131   population:      31,170

-------------------------------------------------------------------------------
  Total Population:      31,170
-------------------------------------------------------------------------------


# 3.5.7 Find all zip codes, with population, within 2 miles of  the Millbrae station

Use the my_station_get_zips() function

In [15]:
my_station_get_zips("Millbrae", 2)


-------------------------------------------------------------------------------
  Zip Codes within 2 mile(s) of Millbrae BART Station
-------------------------------------------------------------------------------

     zip: 94030   population:      22,710
     zip: 94128   population:         135

-------------------------------------------------------------------------------
  Total Population:      22,845
-------------------------------------------------------------------------------
